<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularization

*Author: Alexander Del Toro Barba*

## L1

* xxx


# RNN Model

## Import & Prepare Data

In [1]:
import tensorflow as tf
import datetime, os

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Choose Regularization

**Define lambda**

In [0]:
lamb = 0.0001

**Select Regulizer Type**

The Dense layer takes three regularizers, which all default to None. 

In [0]:
# Regulizer applied to weights
kernel_regularizer=tf.keras.regularizers.l2(l=lamb)

# Regulizer applied to bias unit
# bias_regularizer=

# Regulizer applied to layer activation
# activity_regularizer


## Define Model & Run

In [5]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
model.add(tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=kernel_regularizer))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=5, validation_data=(x_test, y_test))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 83us/sample - loss: 0.5585 - acc: 0.8206 - val_loss: 0.4669 - val_acc: 0.8543
Epoch 2/5
60000/60000 [==============================] - 5s 78us/sample - loss: 0.4566 - acc: 0.8578 - val_loss: 0.4564 - val_acc: 0.8622
Epoch 3/5
60000/60000 [==============================] - 5s 76us/sample - loss: 0.4333 - acc: 0.8643 - val_loss: 0.4450 - val_acc: 0.8583
Epoch 4/5
60000/60000 [==============================] - 5s 79us/sample - loss: 0.4210 - acc: 0.8708 - val_loss: 0.4563 - val_acc: 0.8584
Epoch 5/5
60000/60000 [==============================] - 5s 77us/sample - loss: 0.4089 - acc: 0.8711 - val_loss: 0.4446 - val_acc: 0.8653
